# PPO

## Imports and hyperparameters

In [19]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

import random
import math
import time

import matplotlib.pyplot as plt

from collections import namedtuple

import gymnasium as gym

In [4]:
env = gym.make('CartPole-v1')

N_OBSERVATIONS = env.observation_space.shape[0]
N_ACTIONS = env.action_space.n
N_AGENTS = 2
T = 50
GAMMA = 1.

TEST = True
TRAIN = False

layer_dim = 128

## Model and selecting actions

### Model

I'll use the same parameters for the policy and value networks. If this doesn't work well I can change it later.

In [5]:
class PolicyAndValueNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(N_OBSERVATIONS, layer_dim),
            nn.ReLU(),
            nn.Linear(layer_dim, layer_dim),
            nn.ReLU(),
            nn.Linear(layer_dim, layer_dim),
            nn.ReLU(),
            # +1 comes from value prediction
            nn.Linear(layer_dim, N_ACTIONS + 1)
        )

    def forward(self, x):
        logits = self.network(x)
        return logits

### Select action

Select action based on log probabilities.

In [6]:
def select_action(action_logits):
    action_probs = torch.softmax(action_logits, dim=0)
    action = torch.multinomial(action_probs, num_samples=1).item()
    return action

In [7]:
if TEST:
    test_env = gym.make('CartPole-v1', render_mode='human')
    test_model = PolicyAndValueNetwork()

    for _ in range(1):
        state, info = test_env.reset()
        terminated, truncated = False, False
        while not (terminated or truncated):
            tensor_state = torch.from_numpy(state)
            logits = test_model(tensor_state)
            action_logits = logits[:-1]
            action = select_action(action_logits)
            state, reward, terminated, truncated, info = test_env.step(action)

    test_env.close()

## Data

### Data collector

We first collect the data while marking episode ends when they happen.

In [8]:
class DataCollector():
    def __init__(self, n_agents):
        self.states = [[] for _ in range(n_agents)]
        self.actions = [[] for _ in range(n_agents)]
        self.rewards = [[] for _ in range(n_agents)]
        self.pis = [[] for _ in range(n_agents)]
        self.episode_ends = [[] for _ in range(n_agents)]

    def add_step(self, agent_id, state, action, reward, pi):
        self.states[agent_id].append(state)
        self.actions[agent_id].append(action)
        self.rewards[agent_id].append(reward)
        self.pis[agent_id].append(pi)

    def mark_episode_end(self, agent_id, timestep):
        self.episode_ends[agent_id].append(timestep)

    def fetch_data(self):
        list_of_agent_states = [torch.stack(agent_states) for agent_states in self.states]
        states = torch.stack([agent_states for agent_states in list_of_agent_states])
        actions = self.actions
        rewards = torch.tensor(self.rewards)
        pis = torch.tensor(self.pis)
        episode_ends = self.episode_ends

        return states, actions, rewards, pis, episode_ends

In [15]:
if TEST:
    # Initialize training data, test environment
    test_data_collector = DataCollector(N_AGENTS)
    test_env = gym.make('CartPole-v1')
    test_model = PolicyAndValueNetwork()
    
    for agent_id in range(N_AGENTS):
        state, _ = test_env.reset()
        terminated, truncated = False, False
        
        for step in range(T):
            # Compute and divide model output
            with torch.no_grad():
                state = torch.from_numpy(state)
                output = test_model(state)
            action_logits, value_logit = output[:-1], output[-1]

            # Select and perform action
            action = select_action(action_logits)
            next_state, reward, terminated, truncated, _ = test_env.step(action)

            # Compute pi
            pi = action_logits[action]

            # Store data
            test_data_collector.add_step(agent_id, state, action, reward, pi)

            # If terminated reset env and mark end, otherwise update state
            if terminated or truncated:
                test_data_collector.mark_episode_end(agent_id, step)
                state, _ = test_env.reset()
                terminated, truncated = False, False
            else:
                state = next_state

    states, actions, rewards, pis, episode_ends = test_data_collector.fetch_data()
    print(f"states: {states}")
    print(f"actions: {actions}")
    print(f"rewards: {rewards}")
    print(f"pis: {pis}")
    print(f"episode ends: {episode_ends}")

states: tensor([[[ 3.6938e-02, -3.6720e-02,  4.2823e-02, -1.0182e-02],
         [ 3.6204e-02, -2.3243e-01,  4.2620e-02,  2.9570e-01],
         [ 3.1555e-02, -4.2813e-01,  4.8534e-02,  6.0151e-01],
         [ 2.2993e-02, -2.3372e-01,  6.0564e-02,  3.2450e-01],
         [ 1.8318e-02, -4.2965e-01,  6.7054e-02,  6.3565e-01],
         [ 9.7253e-03, -2.3553e-01,  7.9767e-02,  3.6482e-01],
         [ 5.0148e-03, -4.1622e-02,  8.7064e-02,  9.8316e-02],
         [ 4.1823e-03,  1.5215e-01,  8.9030e-02, -1.6568e-01],
         [ 7.2253e-03,  3.4589e-01,  8.5716e-02, -4.2900e-01],
         [ 1.4143e-02,  5.3970e-01,  7.7136e-02, -6.9348e-01],
         [ 2.4937e-02,  3.4360e-01,  6.3267e-02, -3.7754e-01],
         [ 3.1809e-02,  1.4764e-01,  5.5716e-02, -6.5603e-02],
         [ 3.4762e-02, -4.8235e-02,  5.4404e-02,  2.4413e-01],
         [ 3.3797e-02, -2.4409e-01,  5.9286e-02,  5.5346e-01],
         [ 2.8916e-02, -4.3999e-01,  7.0356e-02,  8.6422e-01],
         [ 2.0116e-02, -2.4589e-01,  8.7640e-02

### Compute advantages and target values

Next we'll need to compute advantages and target values. We'll start from the end of the data for each agent and move backwards. This makes the desired values easier to compute and to take into account episode ends.

Gradients should be on here because I'll use the computations here to optimize the model.

> If the trajectory terminated due to the maximal trajectory length T
being reached, Vωold (st+n ) denotes the state value associated with state st+n as predicted by the state value
network. Otherwise, Vωold (st+n ) is set to 0

Instead of computing r_t + V(s_t+1) I'm using V(s_t). It's similar enough and shouldn't be that much of a problem.

In [10]:
def compute_advantages_and_target_values(model, states, rewards, episode_ends):
    n_agents = len(episode_ends)

    # We'll add them from last to first, then reverse list
    reversed_target_values = [[] for _ in range(n_agents)]
    reversed_advantages = [[] for _ in range(n_agents)]

    for agent in range(n_agents):
        for i in range(T):
            # Get step state and reward and compute predicted value
            state = states[agent, -i-1]
            reward = rewards[agent, -i-1]
            predicted_value = model(state)[-1]

            # If step is final
            if i in episode_ends[agent]:
                target_value = reward
            # If step is last but not final
            elif i == 0:
                target_value = predicted_value
            # non-last non-final step
            else:
                # Get previous target value and compute 
                previous_target_value = reversed_target_values[agent][-1]
                target_value = reward + GAMMA * previous_target_value
            advantage = target_value - predicted_value

            # Append target value and advantage
            reversed_target_values[agent].append(target_value)
            reversed_advantages[agent].append(advantage)

    target_values = [torch.stack(agent_target_values[::-1]) for agent_target_values in reversed_target_values]
    advantages = [torch.stack(agent_advantage[::-1]) for agent_advantage in reversed_advantages]

    return target_values, advantages

In [11]:
if TEST:
    target_values, advantages = compute_advantages_and_target_values(test_model, states, rewards, episode_ends)
    print(f"target values: {target_values}")
    print(f"advantages: {advantages}")

target values: [tensor([ 2.0000e+01,  1.9000e+01,  1.8000e+01,  1.7000e+01,  1.6000e+01,
         1.5000e+01,  1.4000e+01,  1.3000e+01,  1.2000e+01,  1.1000e+01,
         1.0000e+01,  9.0000e+00,  8.0000e+00,  7.0000e+00,  6.0000e+00,
         5.0000e+00,  4.0000e+00,  3.0000e+00,  2.0000e+00,  1.0000e+00,
         2.8972e+01,  2.7972e+01,  2.6972e+01,  2.5972e+01,  2.4972e+01,
         2.3972e+01,  2.2972e+01,  2.1972e+01,  2.0972e+01,  1.9972e+01,
         1.8972e+01,  1.7972e+01,  1.6972e+01,  1.5972e+01,  1.4972e+01,
         1.3972e+01,  1.2972e+01,  1.1972e+01,  1.0972e+01,  9.9725e+00,
         8.9725e+00,  7.9725e+00,  6.9725e+00,  5.9725e+00,  4.9725e+00,
         3.9725e+00,  2.9725e+00,  1.9725e+00,  9.7249e-01, -2.7508e-02],
       grad_fn=<StackBackward0>), tensor([ 1.0000e+00,  1.1000e+01,  1.0000e+01,  9.0000e+00,  8.0000e+00,
         7.0000e+00,  6.0000e+00,  5.0000e+00,  4.0000e+00,  3.0000e+00,
         2.0000e+00,  1.0000e+00,  3.6999e+01,  3.5999e+01,  3.4999e+01,


### PPO training data

I need to modify the data so that I can sample random batches from it.

In [36]:

print(f"states: {states.shape}")
print(f"states: {states.reshape(N_AGENTS * T, -1).shape}")

print(f"actions: {len(actions), len(actions[0])}")
print(f"actions: {len([action for agent in actions for action in agent])}")

print(f"rewards: {rewards.shape}")
print(f"rewards: {rewards.reshape(N_AGENTS * T).shape}")

print(f"pis: {pis.shape}")
print(f"pis: {pis.reshape(N_AGENTS * T).shape}")

print(f"target values: {len(target_values), target_values[0].shape}")
print(f"target values: {torch.cat(target_values).shape}")

print(f"advantages: {len(advantages), advantages[0].shape}")
print(f"advantages: {torch.cat(advantages).shape}")

states: torch.Size([2, 50, 4])
states: torch.Size([100, 4])
actions: (2, 50)
actions: 100
rewards: torch.Size([2, 50])
rewards: torch.Size([100])
pis: torch.Size([2, 50])
pis: torch.Size([100])
target values: (2, torch.Size([50]))
target values: torch.Size([100])
advantages: (2, torch.Size([50]))
advantages: torch.Size([100])


In [39]:
class PPODataset(Dataset):
    def __init__(self, states, actions, pis, target_values, advantages):
        # Reshape all into (N_AGENTS * T, whatever (possibly 0))
        self.states = states.reshape(N_AGENTS * T, -1)
        self.actions = [action for agent in actions for action in agent]
        self.pis = pis.reshape(N_AGENTS * T)
        self.target_values = torch.cat(target_values)
        self.advantages = torch.cat(advantages)
        
    def __len__(self):
        return len(self.actions)
    
    def __getitem__(self, i):
        state = self.states[i]
        action = self.actions[i]
        pi = self.pis[i]
        target_value = self.target_values[i]
        advantage = self.advantages[i]

        return state, action, pi, target_value, advantage

In [13]:
if TEST:
    # Initialize training data, test environment
    test_training_data = TrainingData()
    test_env = gym.make('CartPole-v1')
    test_model = PolicyAndValueNetwork()
    
    state, _ = test_env.reset()
    terminated, truncated = False, False
    
    while not (terminated or truncated):
        # Compute and divide model output
        with torch.no_grad():
            state = torch.from_numpy(state)
            output = test_model(state)
        action_logits, value_logit = output[:-1], output[-1]

        # Select and perform action
        action = select_action(action_logits)
        next_state, reward, terminated, truncated, _ = test_env.step(action)

        # Compute pi
        pi = action_logits

        # Store data
        test_training_data.push(state, action, reward, pi)

        # Update state
        state = next_state

    # Unpack training data
    state, action, reward, pi, _, _ = test_training_data.unpack()
    print(f"state: {state}")
    print(f"action: {action}")
    print(f"reward: {reward}")
    print(f"pi: {pi}")

    # Compute advantage and target value for each transition
    for i in range(len(test_training_data)):
        print(test_training_data.batch_index)
        i_data = test_training_data.fetch_batch(1)
        print(i_data)

TypeError: Transition.__new__() takes 5 positional arguments but 7 were given